# Import librairies

In [12]:
import pandas as pd
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModel, pipeline
from transformers import PLBartForConditionalGeneration, PLBartTokenizer
from transformers import TrainingArguments,Trainer
from datasets import load_dataset

In [11]:
DATA_PATH_1 = "../data/data_pairs_1.csv"
DATA_PATH_2 = "../data/data_pairs_2.csv"

# Import Data

In [34]:
df = pd.read_csv(DATA_PATH_1).drop(['title'],axis=1)
df

,markdown,code
0,"House Prices: Advanced Regression Techniques Predict sales prices and practice feature engineering, RFs, and gradient boosting Weekly updateCompetition DescriptionAsk a home buyer to describe their dream house, and they probably won t begin with the height of the basement ceiling or the proximity to an east west railroad. But this playground competition s dataset proves that much more influences price negotiations than the number of bedrooms or a white picket fence.With 79 explanatory variables describing almost every aspect of residential homes in Ames, Iowa, this competition challenges y...",import numpy as np
1,"data processing, CSV file I O e.g. pd.read csv",import pandas as pd
2,"Input data files are available in the .. input directory. For example, running this by clicking run or pressing Shift Enter will list the files in the input directory",from datetime import datetime
3,for some statistics,"from scipy.stats import skew from scipy.special import boxcox1p from scipy.stats import boxcox_normmax from sklearn.linear_model import ElasticNetCV , LassoCV , RidgeCV from sklearn.ensemble import GradientBoostingRegressor from sklearn.svm import SVR from sklearn.pipeline import make_pipeline from sklearn.preprocessing import RobustScaler from sklearn.model_selection import KFold , cross_val_score from sklearn.metrics import mean_squared_error from mlxtend.regressor import StackingCVRegressor from xgboost import XGBRegressor from lightgbm import LGBMRegressor import matplotli..."
4,Any results you write to the current directory are saved as output. Load data,"train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')\r\ntest = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')\r\nprint (""Data is loaded!"") print (""Train: "",train.shape[0],""sales, and "",train.shape[1],""features"")\r\nprint (""Test: "",test.shape[0],""sales, and "",test.shape[1],""features"") train.head() test.head()"
...,...,...
7567,"If the word exists in the vocabulary, this command prints a number N, meaning that the Nth feature of the matrix is that word.",test_features = tfidf.transform(X_test).toarray()\r\nprint(test_features.shape) train_labels = Y_train\r\ntest_labels = Y_test
7568,"reduce the matrix dimensionality in order to reduce the dimensionality of our matrix ! Feature matrix shape: Number of documents x Length of vocabulary we can carry out some Feature Selection, the process of selecting a subset of relevant variables. I will proceed as follows:treat each category as binary for example, the Tech category is 1 for the Tech news and 0 for the others perform a Chi Square test to determine whether a feature and the binary target are independent keep only the features with a certain p value from the Chi Square test.","""""""from sklearn import feature_selection \r\ny = data_clean['target']\r\nX_names = tfidf.get_feature_names()\r\np_value_limit = 0.95\r\ndtf_features = pd.DataFrame()\r\nfor cat in np.unique(y):\r\n chi2, p = feature_selection.chi2(X_train, y==cat)\r\n dtf_features = dtf_features.append(pd.DataFrame(\r\n {""feature"":X_names, ""score"":1-p, ""y"":cat}))\r\n dtf_features = dtf_features.sort_values([""y"",""score""], \r\n ascending=[True,False])\r\n dtf_features = dtf_features[dtf_features[""score""]>p_value_limit]\r\nX_names = dtf_features[""feature""].unique().tolist()\r\nlen(X_names)"""""""
7569,Model training Model : Multinomial NB,"import pandas as pd\r\nfrom sklearn.naive_bayes import MultinomialNB\r\nfrom sklearn.metrics import classification_report, confusion_matrix, accuracy_score mnb_classifier = MultinomialNB() mnb_classifier.fit(train_features,train_labels) mnb_prediction = mnb_classifier.predict(test_features)"
7570,Visualizing scikit model performance,"training_accuracy = accuracy_score(train_labels, mnb_classifier.predict(train_features))\r\nprint(training_accuracy) testing_accuracy = accuracy_score(test_labels, mnb_prediction)\r\nprint(testing_acc

# Import the pretrained model "PLBART" and its tokenizer

In [14]:
plbarttokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="en_XX")
plbart_model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")

# Tokenize the dataset

In [15]:
def fn_tokenize(data):
    new_df = pd.DataFrame(columns=['input_ids_markdown','attention_mask_markdown','input_ids_code','attention_mask_code'])
    for i in data.index:
        new_df.loc[len(new_df)] = [plbarttokenizer(data.loc[i]['markdown'])['input_ids'],plbarttokenizer(data.loc[i]['markdown'])['attention_mask'],plbarttokenizer(data.loc[i]['code'])['input_ids'],plbarttokenizer(data.loc[i]['code'])['attention_mask']]

    return new_df

In [16]:
new_df_1 = fn_tokenize(df_1)
new_df_1

,input_ids_markdown,attention_mask_markdown,input_ids_code,attention_mask_code
0,"[26343, 5544, 1118, 33475, 18938, 30816, 21498, 15521, 15485, 573, 13333, 18047, 135, 6387, 2118, 3478, 16069, 33463, 246, 12544, 33463, 135, 8169, 12380, 45, 20077, 185, 915, 2489, 13559, 9427, 32187, 14, 5072, 15328, 10, 71, 6734, 2614, 36, 562, 15476, 33463, 135, 1534, 2925, 2482, 7, 2995, 215, 57, 1636, 153, 57, 1314, 228, 33298, 255, 57, 23359, 30784, 71, 197, 21476, 23965, 622, 89, 59, 110, 33455, 1160, 143, 1772, 1550, 718, 13559, 11, 4092, 1426, 29, 208, 2449, 1010, 3134, 7191, 2210, 4530, 8789, 97, 15529, 1433, 57, 1019, 153, 25397, 27152, 255, 14, 5890, 3366, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[662, 2988, 268, 865, 2, 50002]","[1, 1, 1, 1, 1, 1]"
1,"[272, 5037, 33463, 6423, 375, 34, 261, 168, 33455, 33461, 33455, 3880, 33455, 474, 3771, 2, 50002]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[662, 4112, 268, 3880, 2, 50002]","[1, 1, 1, 1, 1, 1]"
2,"[2782, 272, 1139, 395, 2519, 55, 57, 1269, 641, 2139, 33455, 1276, 812, 33463, 1826, 143, 389, 6148, 521, 255, 14630, 23549, 7645, 481, 410, 57, 1139, 55, 57, 641, 2139, 2, 50002]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[320, 2220, 662, 2220, 2, 50002]","[1, 1, 1, 1, 1, 1]"
3,"[126, 632, 11182, 2, 50002]","[1, 1, 1, 1, 1]","[320, 9021, 33455, 6300, 662, 24130, 320, 9021, 33455, 11716, 662, 2534, 33449, 1268, 33485, 33453, 320, 9021, 33455, 6300, 662, 2534, 33449, 1268, 33456, 8717, 2411, 320, 13446, 33455, 11244, 33456, 1856, 662, 20642, 7760, 8665, 16, 282, 19168, 8665, 16, 246, 33340, 8665, 320, 13446, 33455, 2682, 33457, 136, 662, 19077, 20064, 45, 22709, 320, 13446, 33455, 28146, 662, 6737, 33487, 320, 13446, 33455, 15645, 662, 797, 33456, 15645, 320, 13446, 33455, 1800, 12626, 662, 12054, 347, 28267, 320, 13446, 33455, 1856, 33456, 10086, 662, 1202, 18639, 16, 5542, 33456, 465, 33456, 5034, 320, 13446, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
4,"[1916, 1487, 144, 661, 71, 57, 759, 2139, 395, 3410, 268, 699, 33455, 5012, 272, 2, 50002]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3370, 24, 3880, 33455, 474, 33456, 4424, 27716, 1216, 33488, 9118, 33489, 32425, 33489, 26087, 33489, 24018, 33489, 28579, 15521, 33488, 5259, 33455, 4424, 1156, 242, 24, 3880, 33455, 474, 33456, 4424, 27716, 1216, 33488, 9118, 33489, 32425, 33489, 26087, 33489, 24018, 33489, 28579, 15521, 33488, 953, 33455, 4424, 1156, 597, 6368, 588, 96, 3331, 23669, 597, 6368, 22012, 33475, 8248, 5259, 33455, 2788, 9892, 33474, 21167, 33463, 135, 8248, 5259, 33455, 2788, 13911, 33474, 6160, 1191, 597, 6368, 866, 33475, 8248, 953, 33455, 2788, 9892, 33474, 21167, 33463, 135, 8248, 953, 33455, 2788, 1391...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
...,...,...,...,...
7567,"[708, 57, 1677, 2189, 55, 57, 18686, 33463, 143, 1130, 6078, 14, 1019, 171, 33463, 7464, 208, 57, 171, 155, 2118, 153, 57, 3767, 96, 208, 1677, 33455, 2, 50002]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[242, 33456, 6160, 24, 2730, 20667, 33455, 5427, 33460, 33522, 33456, 953, 426, 545, 1701, 292, 597, 33460, 953, 3345

In [8]:
new_df_1 = new_df_1.sample(frac = 1).reset_index(drop=True)

In [9]:
new_df_1[5600:].to_csv("../data/test_tokenized_data_1.csv",index=False)
new_df_1[:5600].to_csv("../data/train_tokenized_data_1.csv",index=False)

In [10]:
new_df_1.to_csv("../data/tokenized_data_1.csv")

In [12]:
dataset_1 =  load_dataset("csv", data_files={"train": ["../data/train_tokenized_data_1.csv"], "test": "../data/test_tokenized_data_1.csv"})

Using custom data configuration default-07e9f75700f59e54


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\zdahmani\.cache\huggingface\datasets\csv\default-07e9f75700f59e54\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
small_train_dataset = dataset_1["train"].shuffle(seed=42)
small_eval_dataset = dataset_1["test"].shuffle(seed=42)

Loading cached shuffled indices for dataset at C:\Users\zdahmani\.cache\huggingface\datasets\csv\default-07e9f75700f59e54\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-f59cb5662487984f.arrow
Loading cached shuffled indices for dataset at C:\Users\zdahmani\.cache\huggingface\datasets\csv\default-07e9f75700f59e54\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-130bd719c2a31e29.arrow


In [23]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [22]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [33]:
trainer = Trainer(
    model=plbart_model,
    args=training_args,
    train_dataset=small_train_dataset,
     #eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `PLBartForConditionalGeneration.forward` and have been ignored: attention_mask_markdown, input_ids_code, attention_mask_code, input_ids_markdown. If attention_mask_markdown, input_ids_code, attention_mask_code, input_ids_markdown are not expected by `PLBartForConditionalGeneration.forward`,  you can safely ignore this message.
c:\Users\zdahmani\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5600
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2

  0%|          | 0/2100 [00:00<?, ?it/s]

ZeroDivisionError: integer division or modulo by zero

Another Method

In [33]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [35]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [36]:
tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="en_XX")
MAX_LEN = 512
SUMMARY_LEN = 150
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-4

In [41]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.markdown = self.data.markdown
        self.code = self.data.code

    def __len__(self):
        return len(self.markdown)

    def __getitem__(self, index):
        code = str(self.code[index])
        code = ' '.join(code.split())

        markdown = str(self.markdown[index])
        markdown = ' '.join(markdown.split())

        source = self.tokenizer.batch_encode_plus([code], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([markdown], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [42]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

FULL Dataset: (7572, 2)
TRAIN Dataset: (6058, 2)
TEST Dataset: (1514, 2)


In [43]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [44]:
model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")
model = model.to(device)

In [45]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [54]:
def train(epoch):
    c = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        #lm_labels = y[:, 1:].clone().detach()
        #lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100

        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids)
        loss = outputs[0]
        
        if _%500==0:
            #print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            c+=1
        
        optimizer.zero_grad()
        loss.sum().backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()
    print(c)

In [55]:
for epoch in range(EPOCHS):
    train(epoch)

c:\Users\zdahmani\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


7


In [ ]:
def writer(predictions, actuals):
    # output_path = r'/content/drive/My Drive/Colab Notebooks/'
    # pred_file = output_path+'preds.txt'
    # actual_file = output_path+'actuals.txt'
    # with open(pred_file, 'w') as f:
    #     for pred in predictions:
    #         f.write('%s\n' % pred)

    # with open(actual_file, 'w') as f:
    #     for actual in actuals:
    #         f.write('%s\n' % actual)
    my_dict = {
        'Actual Text': actuals,
        'Prediction': predictions       
        }
    final_output = pd.DataFrame(my_dict)

    return final_output

In [ ]:
def validate(epoch):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
for epoch in range(1):
    predictions, actuals = validate(epoch)
    final_df = writer(predictions, actuals)
    final_df.to_csv('../data/data.csv')
    print('Output Files generated for review')